In [ ]:
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} pygeos

In [47]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [2]:
# Datasets
LTS = gpd.read_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_shape\LTS_subset.shp')
BDF5L = gpd.read_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\BFDL_ML46\BFD5L_subset.shp')
NDVI_10Dmax = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\NDVI_10Dmax_1km.nc')
RADOLAN = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\RADOLAN_1km.nc')
SSM = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\SSM_1km_V1.nc')

In [15]:
# Define a point of interest (x,y), so (longitude, latitude).
point = (8.51559, 49.7429) #LTS ID 42142960

In [16]:
# Create a DataFrame from point of interest.
p_df = pd.DataFrame.from_dict({'x':[8.51559], 'y':[49.7429]})

In [24]:
# Create GeoDataFrame from DataFrame.
p_gdf = gpd.GeoDataFrame(p_df, geometry=gpd.points_from_xy(p_df.x, p_df.y), crs='EPSG:4326')

In [25]:
## Table data query.
# Query LTS data.
p_gdf = p_gdf.sjoin_nearest(LTS, lsuffix='LTS_left', rsuffix='LTS_right', distance_col='LTS_distance')
# Query BDF5L data.
p_gdf = p_gdf.sjoin_nearest(BDF5L, lsuffix='BDF5L_left', rsuffix='BDF5L_right', distance_col='BDF5L_distance')

C:\Users\USER\Anaconda3\envs\irrigation_detection\lib\site-packages\geopandas\array.py:341: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [100]:
## Grid data query.
test = SSM.sel(lon=point[0], lat=point[1], method='nearest')
test = test.where(test.ssm != np.nan, drop=True)
test.ssm.hvplot.line(groupby='time.year')

:DynamicMap   [time.year]
   :Curve   [time]   (ssm)

In [73]:
# Plot GeoDataFrame table.
p_gdf.hvplot.table(columns=[x for x in list(p_gdf.columns.values) if x != 'geometry'])

:Table   [x,y,index_LTS_right,POINT_ID,coarse,clay,silt,sand,pH_in_H2O,pH_in_CaCl,OC,CaCO3,N,P,K,CEC,Notes,sample_ID,GPS_LAT,GPS_LONG,Elevation,LC1,LU1,Soil_Stone,NUTS_0,NUTS_1,NUTS_2,NUTS_3,LC1_Desc,LU1_Desc,LTS_distance,index_BDF5L_right,OBJECTID,GMKNR,WERT,BODENART,SHAPE_Leng,SHAPE_Area,BDF5L_distance]